FOR THREE LAnguage as we discussed we are going to implement English,Hindi,gujrati

In [1]:
import numpy as np
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import pickle
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Amit
[nltk_data]     Malik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
file = open("Hindi_Stopwords.txt", encoding = "UTF-8")
stopWords = file.read().split("\n")

In [8]:
paras_e = open('Geeta_English.txt', encoding="UTF8").read().split("\n")
paras_h = open('gita_Hindi.txt', encoding="UTF8").read().split("\n")
paras_g = open('gita-gujrati.txt', encoding="UTF8").read().split("\n")


Stemming and tokensing Hindi words

In [9]:
from indicnlp.tokenize import indic_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [10]:
suffixes = {
    1: ["ो", "े", "ू", "ु", "ी", "ि", "ा"],
    2: ["कर", "ाओ", "िए", "ाई", "ाए", "ने", "नी", "ना", "ते", "ीं", "ती", "ता", "ाँ", "ां", "ों", "ें"],
    3: ["ाकर", "ाइए", "ाईं", "ाया", "ेगी", "ेगा", "ोगी", "ोगे", "ाने", "ाना", "ाते", "ाती", "ाता", "तीं", "ाओं", "ाएं", "ुओं", "ुएं", "ुआं"],
    4: ["ाएगी", "ाएगा", "ाओगी", "ाओगे", "एंगी", "ेंगी", "एंगे", "ेंगे", "ूंगी", "ूंगा", "ातीं", "नाओं", "नाएं", "ताओं", "ताएं", "ियाँ", "ियों", "ियां"],
    5: ["ाएंगी", "ाएंगे", "ाऊंगी", "ाऊंगा", "ाइयाँ", "ाइयों", "ाइयां"],
}

def stemming_hindi(text):
    for L in 5, 4, 3, 2, 1:
        if len(text) > L + 1:
            for suf in suffixes[L]:
                if text.endswith(suf):
                    return text[:-L]
    return text

In [11]:
def preprocess_text(string, stopWords):
    punctuations=["।",";",",",":","!",'"',"?",":-","-","{","(","}",")","_","०","S","―","=","[","]","......",":-",".","॥",'”',"|","“","'"]
    string = "".join([w if w not in punctuations else " " for w in string])  #To remove punctuations
    tokens = string.split()
    tokens = [stemming_hindi(word) for word in tokens if word not in stopWords]
    tokens = [w for w in tokens if w not in stopWords]
    return tokens

In [12]:
pos_lis = {}
for idx, para in enumerate(paras_h):
    words = preprocess_text(para, stopWords)
    
    for word in words:
        if word in pos_lis.keys():
            if idx in pos_lis[word].keys():
                pos_lis[word][idx] += 1
            else:
                pos_lis[word][idx] = 1
        else:
            temp = {idx : 1}
            pos_lis[word] = temp.copy()

BM25

In [13]:
def BM25(query, pos_lisl, l = 5, b = 0.75, k = 2):
    q_tokens = preprocess_text(query,stopWords)
    lengths = {}
    N = len(paras_e)
    avg_len = 0
    for idx, para in enumerate(paras_e):
        lengths[idx] = len(para)             #cal no of words of each file
        avg_len += lengths[idx]
    avg_len /= N
    #Calculate idf of each token
    idf = {}
    for word in np.unique(q_tokens):
        if word in pos_lisl.keys():
            df = len(pos_lisl[word].keys())
        else:
            df = 0
        idf[word] = np.log((N - df + 0.5) / (df + 0.5))
    score = {}
    for idx, para in enumerate(paras_e):
        s = 0
        for word in np.unique(q_tokens):
            tf = 0
            if word in pos_lisl.keys() and idx in pos_lisl[word].keys():
                tf = pos_lisl[word][idx]
            s += idf[word] * (tf * (k + 1)) / (k*(1 - b + b*lengths[idx]/avg_len) + tf)
        score[idx] = s
    return sorted(score, key = score.get, reverse=True)[:l]

Searching and Evaluating 

In [14]:
from googletrans import Translator

def get_translation(data, dest):
    translator = Translator()
    text = translator.translate(data, dest).text
    return text

In [15]:
slokas = open("Shlokas.txt",encoding="UTF-8").read().split("\n")

In [16]:
def search(query, pos_lis, lang, l = 5):
    query = get_translation(query, "hi")
    tokens = preprocess_text(query, stopWords)
    
    query = " ".join(tokens)

    indxs = BM25(query, pos_lis, l)
    print(indxs)
    if lang == 'e':
        return [f'{slokas[idx]}{paras_e[idx]}' for idx in indxs], indxs
    elif lang== 'g':
        return [f'{slokas[idx]}{paras_g[idx]}' for idx in indxs], indxs

    else:
        return [f'{slokas[idx]}{paras_h[idx]}' for idx in indxs], indxs

In [17]:
query ="That gives itself to follow shows of sense"
search(query,pos_lis,"g")

[143, 167, 193, 100, 112]


(['अर्जुन उवाच ।અર્જુને કહ્યું.',
  'सेनयोरुभयोर्मध्ये विषीदन्तमिदं वचः ॥ નિરાશામાં ડૂબી ગયેલી બે સેનાઓ વચ્ચે આ શબ્દો સંભળાયા.',
  'न जायते म्रियते वा कदाचिन्તે ક્યારેય જન્મતો નથી કે મૃત્યુ પામતો નથી',
  'यद्यप्येते न पश्यन्ति लोभोपहतचेतसः ।જો કે આ લોકો જોતા નથી, તેમ છતાં તેમના મન લોભથી કાબુમાં હોય છે.',
  ''],
 [143, 167, 193, 100, 112])

TF-IDF model 

In [21]:
from collections import defaultdict

lengths = defaultdict(int)
for idx, para in enumerate(paras_h, start=1):
    lengths[idx] = len(para)

idf = {}
N = len(slokas)
for word in pos_lis.keys():
    df = len(pos_lis[word].keys())
    idf[word] = np.log((N) / (df))

# Calculate tf-idf score vector of each file
norm_list = defaultdict(list)
for idx, para in enumerate(paras_h, start=1):
    for word in np.unique(para.split()):
       tf = 0
       if word in idf:
            try:
                if idx in pos_lis[word].keys():
                    tf = pos_lis[word][idx] / lengths[idx]
            except KeyError:
                pass
            norm_list[idx].append(tf * idf[word])

for idx in range(1, N):
    norm_list[idx] = np.linalg.norm(norm_list[idx])


In [22]:
def find_cosine_sim(n_docs, query_vector, doc_vectors):
    cosine_sims = []
    for i in range(n_docs-1):
        dot = np.dot(query_vector, doc_vectors[i])
        query_norm = np.linalg.norm(query_vector)
        doc_norm = norm_list[i]
        cosine_sims.append(dot/((query_norm + 0.5)*(doc_norm + 0.5)))
    return cosine_sims

In [23]:
def tfidf(query, main_dict):
    n_slokas = len(slokas)
    # finding document vectors
    tfidf_dict = {} # dictionary which stores tfidf of each document
    doc_vectors = [] 
    for i in range(n_slokas):
        n_words = len(paras_h[i].split())
        vector = []
        for word in np.unique(query):
            if word in main_dict.keys() and i in main_dict[word].keys():
#                 tf = main_dict[word][i]/n_words
                tf = main_dict[word][i]
                idf = np.log((n_slokas+1)/(len(main_dict[word].keys())+1))
                tf_idf = tf*idf
                vector.append(tf_idf)
            else:
                vector.append(0)
        doc_vectors.append(vector)
    word_freq_query_dict = Counter(query)
    # finding query vector
    query_vector = []
    # remember np.unique return result in alphabetical order
    for word in np.unique(query):
        if word in main_dict.keys():
#             tf = word_freq_query_dict[word]/len(query)
            tf = word_freq_query_dict[word]
            idf = np.log((n_slokas+1)/(len(main_dict[word].keys())+1))
            tf_idf = tf*idf
            query_vector.append(tf_idf)
        else:
               query_vector.append(0)
#     print(query_vector)
    scores = find_cosine_sim(n_slokas, query_vector, doc_vectors)
    return sorted(np.argsort(np.array(scores))[-5:], reverse=False)


In [25]:
def search(query, pos_lis, lang, l = 5):
    query = get_translation(query, "hi")
    tokens = preprocess_text(query, stopWords)
    
    query = " ".join(tokens)

    indxs = BM25(query, pos_lis, l)
    print(indxs)
    if lang == 'e':
        return [f'{slokas[idx]}{paras_e[idx]}' for idx in indxs], indxs
    elif lang== 'g':
        return [f'{slokas[idx]}{paras_g[idx]}' for idx in indxs], indxs

    else:
        return [f'{slokas[idx]}{paras_h[idx]}' for idx in indxs], indxs

In [27]:
query ="That gives itself to follow shows of sense"
search(query,pos_lis,"e")

[143, 167, 193, 100, 112]


(['अर्जुन उवाच ।',
  'सेनयोरुभयोर्मध्ये विषीदन्तमिदं वचः ॥ Arjuna.',
  "न जायते म्रियते वा कदाचिन्While the Prince wept despairing 'twixt those hosts,",
  'यद्यप्येते न पश्यन्ति लोभोपहतचेतसः ।Triumph and domination, wealth and ease,',
  ''],
 [143, 167, 193, 100, 112])

EVALUATION OF BM25 AND TF-IDF MODEL

In [2]:
!pip install rank_bm25

In [4]:

from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_models(documents, queries, relevant_docs, model_type):
   
    if model_type == 'tfidf':
        vectorizer = TfidfVectorizer()
    elif model_type == 'bm25':
        tokenized_docs = [doc.split() for doc in documents]
        bm25 = BM25Okapi(tokenized_docs)
    else:
        raise ValueError('Model type must be "bm25" or "tfidf"')
    
    
    if model_type == 'tfidf':
        doc_vectors = vectorizer.fit_transform(documents)
    elif model_type == 'bm25':
        pass 
   
    query_vectors = vectorizer.transform(queries)
    similarities = cosine_similarity(query_vectors, doc_vectors)
    num_queries = len(queries)
    avg_precision = 0
    avg_recall = 0
    avg_f1 = 0
    
    for i in range(num_queries):
        retrieved_docs = similarities[i].argsort()[::-1] 
        relevant = set(relevant_docs[i])
        retrieved = set(retrieved_docs[:len(relevant_docs[i])])
        true_positives = relevant.intersection(retrieved)
        false_positives = retrieved - relevant
        false_negatives = relevant - retrieved
        
        precision = len(true_positives) / (len(true_positives) + len(false_positives))
        recall = len(true_positives) / (len(true_positives) + len(false_negatives))
        f1 = 2 * (precision * recall) / (precision + recall)
        
        avg_precision += precision
        avg_recall += recall
        avg_f1 += f1
    
    avg_precision /= num_queries
    avg_recall /= num_queries
    avg_f1 /= num_queries
    
    return avg_precision, avg_recall, avg_f1
